In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from pyproj import Proj, transform, CRS

import requests
import json

from time import sleep 
from tqdm import tqdm

# Berechnung der Entferunung von Gebäude zu Apotheke mithilfe der OSM Routing API 

In [ ]:
bevölkerungs_gdf = gpd.read_file('./WLP/pharmacy_assigned.gpkg')
bevölkerungs_gdf['Alter'] = bevölkerungs_gdf['Alter'].apply(json.loads)
bevölkerungs_gdf['Geschlecht'] = bevölkerungs_gdf['Geschlecht'].apply(json.loads)

pharmacy_df = pd.read_csv('./OSM_Data/Donner-Apotheken.csv')
pharmacy_gdf = gpd.GeoDataFrame(pharmacy_df, geometry=gpd.points_from_xy(pharmacy_df['lon'], pharmacy_df['lat']), crs=CRS("EPSG:4326"))
pharmacy_gdf = pharmacy_gdf.to_crs(bevölkerungs_gdf.crs)

In [ ]:
# Funktion zur Berechnung der Entfernung zur nächstgelegenen Apotheke
def calculate_distance_to_nearest_pharmacy(row):
    # Finden der nächstgelegenen Apotheke basierend auf der zugewiesenen Apotheken-ID
    nearest_pharmacy = pharmacy_gdf[pharmacy_gdf['id'] == row.assigned_pharmacy]

    # Extrahieren der Längen- und Breitengrade des Gebäudes und der Apotheke
    lon_build = row.lon
    lat_build = row.lat
    lon_pharm = nearest_pharmacy.lon.iloc[0]
    lat_pharm = nearest_pharmacy.lat.iloc[0]

    max_retries = 3  # Maximale Anzahl von Wiederholungsversuchen festlegen

    # Schleife für Wiederholungsversuche
    for attempt in range(1, max_retries + 1):
        try:
            # Erstellen der Anforderungs-URL für die OSRM-Route
            request_string = f"http://router.project-osrm.org/route/v1/driving/{lon_build},{lat_build};{lon_pharm},{lat_pharm}?overview=false"
            res = requests.get(request_string)
            res.raise_for_status()  # Ausnahme bei nicht-200-Statuscodes auslösen

            # Inhalt der Antwort laden und die Entfernung und Dauer extrahieren
            content = json.loads(res.content)
            distance = content['routes'][0]['legs'][0]['distance']
            duration = content['routes'][0]['legs'][0]['duration']
            return distance, duration

        except requests.exceptions.RequestException as e:
            # Fehlerbehandlung und Wartezeit vor erneutem Versuch
            print(f"Error occurred on attempt {attempt} for {lon_build},{lat_build} -> {lon_pharm},{lat_pharm}: {e}")
            sleep(30)  # 30 Sekunden warten vor erneutem Versuch

    # Wenn alle Wiederholungsversuche fehlschlagen, 0 für Entfernung und Zeit zurückgeben
    print(f"All retries failed for {lon_build},{lat_build} -> {lon_pharm},{lat_pharm}")
    return 0, 0

In [ ]:
# Aufteilen des DataFrames in 10 gleiche Teile (Chunks)
chunksize = int(len(bevölkerungs_gdf) / 10)
chunks = [bevölkerungs_gdf.iloc[i * chunksize:(i + 1) * chunksize] for i in range(10)]
start_chunk = 0  # Start-Chunk-Index setzen

# Iteration über jeden Chunk ab dem start_chunk-Index
for iteration, chunk in enumerate(chunks[start_chunk:]):
    # Berechnung der Entfernung und Zeit für jede Zeile im Chunk
    for index, row in tqdm(chunk.iterrows(), total=len(chunk)):
        distance, time = calculate_distance_to_nearest_pharmacy(row)
        chunk.loc[index, 'distance_pharmacy'] = distance
        chunk.loc[index, 'time_pharmacy'] = time

    # Konvertierung spezifischer Spalten in JSON und Speichern des Chunks
    chunk['Alter'] = chunk['Alter'].apply(json.dumps)
    chunk['Geschlecht'] = chunk['Geschlecht'].apply(json.dumps)
    chunk.to_file(f'./Processed_WLP/Würzburg/pharmacy_assigned_{iteration + start_chunk}.gpkg', driver='GPKG')

print("All chunks processed and saved successfully!")

In [ ]:
# Berechnung der Chunkgröße und des Restes
chunksize = len(bevölkerungs_gdf) // 10  # Bestimmen der Anzahl der Zeilen pro Chunk
remainder = len(bevölkerungs_gdf) % 10  # Bestimmen der Anzahl der verbleibenden Zeilen

# Verarbeitung der verbleibenden Zeilen (falls vorhanden)
if remainder > 0:
    last_chunk_start = 10 * chunksize  # Startindex für den letzten Chunk
    last_chunk = bevölkerungs_gdf.iloc[last_chunk_start:]  # Extrahieren des letzten Chunks

    # Berechnung der Entfernung und Zeit für jede Zeile im letzten Chunk
    for index, row in tqdm(last_chunk.iterrows(), total=len(last_chunk)):
        distance, time = calculate_distance_to_nearest_pharmacy(row)
        last_chunk.loc[index, 'distance_pharmacy'] = distance
        last_chunk.loc[index, 'time_pharmacy'] = time
    
    # Konvertierung der 'Alter'- und 'Geschlecht'-Spalten in JSON-Format
    last_chunk['Alter'] = last_chunk['Alter'].apply(json.dumps)
    last_chunk['Geschlecht'] = last_chunk['Geschlecht'].apply(json.dumps)
    
    # Speichern des letzten Chunks als GeoPackage-Datei
    last_chunk.to_file(f'./Processed_WLP/Würzburg/pharmacy_assigned_10.gpkg', driver='GPKG')
    print("Processed remaining rows")

In [ ]:
# Liste zur Speicherung der Dateipfade
file_paths = []
# Erstellen der Dateipfade für die 11 GeoPackage-Dateien
for i in range(11):
    file_paths.append(f'./Processed_WLP/Würzburg/pharmacy_assigned_{i}.gpkg')

# Verkettung der GeoDataFrames mittels Listenverständnis
bevölkerungs_gdf = pd.concat([gpd.read_file(path) for path in file_paths])

# Speichern des zusammengefügten GeoDataFrames als eine vollständige GeoPackage-Datei
bevölkerungs_gdf.to_file('./WLP/pharmacy_assigned_complete.gpkg', driver='GPKG')